## Calculate climate indicies from surface temperatures

Climate indicies like the [NINO3.4 index](https://climatedataguide.ucar.edu/climate-data/nino-sst-indices-nino-12-3-34-4-oni-and-tni) are commonly used for analyzing climate variations from observations and models. xarrayutils contains some functions that make it easy to calculate climate indicies from any xarray dataset.

In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
%matplotlib inline

The NINO3.4 index is defined on sea surface temperature (SST) data. Lets use the NOAA ERSST v5 product to recreate the index:

In [13]:
# load SST
sst = xr.open_dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc',
                      chunks={'time':1}, use_cftime=True)['sst']
# sort the latitude (this is a quirk of the dataset and not always needed)
sst = sst.sortby('lat').sel(lat=slice(-10, 10), time=slice('1960', None))

# usually we would want to use the actual area, but to keep it simple we just use ones here.
sst.coords['area'] = xr.ones_like(sst.isel(time=20).drop('time'))
sst

<xarray.DataArray 'sst' (time: 719, lat: 11, lon: 180)>
dask.array<getitem, shape=(719, 11, 180), dtype=float32, chunksize=(1, 11, 180), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -10.0 -8.0 -6.0 -4.0 -2.0 0.0 2.0 4.0 6.0 8.0 10.0
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * time     (time) object 1960-01-01 00:00:00 ... 2019-11-01 00:00:00
    area     (lat, lon) float32 dask.array<chunksize=(11, 180), meta=np.ndarray>
Attributes:
    long_name:     Monthly Means of Sea Surface Temperature
    units:         degC
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    dataset:       NOAA Extended Reconstructed SST V5
    parent_stat:   Individual Values
    actual_range:  [-1.8     42.32636]
    valid_range:   [-1.8 45. ]
    _ChunkSizes:   [  1  89 180]

In [14]:
from xarrayutils.climate_indicies import extract_climate_indicies

boxes = {'NINO3.4': {'lon': slice(190, 240), 'lat': slice(-5, 5)}}

# Since the data is downloaded from the opendap link on the fly, this can take a bit of time,
# depending on the internet connection speed
from dask.diagnostics import ProgressBar
with ProgressBar():
    NINO = extract_climate_indicies(sst,
                                    xdim='lon',
                                    ydim='lat',
                                    boxes=boxes,
                                    print_map=True,
                                    area_coord='area',
                                    clim_period=slice('1960','1989')
                                   )

[#############                           ] | 33% Completed |  0.1s

/Users/juliusbusecke/code/xarrayutils/xarrayutils/climate_indicies.py:165: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax = plt.axes(projection=ccrs.Robinson(180))


[########################################] | 100% Completed |  0.3s
Calculating NINO3.4 index
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  2min 18.2s


In [10]:
NINO['NINO3.4']#.plot()

<xarray.DataArray 'NINO3.4' (time: 1991)>
array([-0.46134233, -0.2749826 , -0.11805411, ...,  0.70969564,
        0.6486169 ,  0.7148889 ], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1854-01-01 1854-02-01 ... 2019-11-01
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 12 ... 1 2 3 4 5 6 7 8 9 10 11

In [2]:
url = 'http://iridl.ldeo.columbia.edu/SOURCES/.NASA/.GPCP/.V2p1/.multi-satellite/.prcp/dods'
dsp = xr.open_dataset(url, decode_times=False)
# fix IRIDL metadata that does not conform to standards
dsp.T.attrs['calendar'] = '360_day'
dsp = xr.decode_cf(dsp)
dsp

<xarray.Dataset>
Dimensions:  (T: 360, X: 144, Y: 72)
Coordinates:
  * X        (X) float32 1.25 3.75 6.25 8.75 ... 351.25 353.75 356.25 358.75
  * T        (T) object 1979-01-16 00:00:00 ... 2008-12-16 00:00:00
  * Y        (Y) float32 88.75 86.25 83.75 81.25 ... -81.25 -83.75 -86.25 -88.75
Data variables:
    prcp     (T, Y, X) float32 ...
Attributes:
    Conventions:  IRIDL

In [4]:
# !cd ..; pip install -e .

ModuleNotFoundError: No module named 'scipy'